In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
import sys
import os
import pickle

# Add the parent directory of 'src' to the Python path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# Load environment variables from .env file
load_dotenv()
from src.config_loader import config
from src.utils import create_html_story, create_test_story_dict



Found config file at: y:\Python Scripts\audio-language-trainer\src\config.json
Language name: Swedish determined from code sv
Successfully loaded config from: y:\Python Scripts\audio-language-trainer\src\config.json
Multiple country codes available for en: en-AU, en-GB, en-IN, en-US


In [50]:
with open("../outputs/a_rose_in_the_office\story_data_a_rose_in_the_office.json.pkl", "rb") as file:
    story_data = pickle.load(file)

In [51]:
story_data = create_test_story_dict(story_data, 1, 1)

In [9]:
from src.anki_tools import generate_wiktionary_links_non_english


def test_wiktionary_links():
    """Test function to demonstrate usage."""
    test_cases = [
        ("goodbye England", "uk"),  # Ukrainian
        ("book reading", "sv"),  # Swedish
        ("coffee shop", "ja"),  # Japanese
    ]
    
    for phrase, lang_code in test_cases:
        print(f"\nTesting {lang_code} Wiktionary links for: {phrase}")
        result = generate_wiktionary_links_non_english(phrase, lang_code)
        print(f"Result: {result}")

In [13]:
test_wiktionary_links()


Testing uk Wiktionary links for: goodbye England
native url is: https://uk.wiktionary.org/wiki/goodbye
native url is: https://uk.wiktionary.org/wiki/england
Result: <a href="https://uk.wiktionary.org/wiki/goodbye#Англійська">goodbye</a> <a href="https://en.wiktionary.org/wiki/england#English">England</a>

Testing sv Wiktionary links for: book reading
native url is: https://sv.wiktionary.org/wiki/book
native url is: https://sv.wiktionary.org/wiki/reading
Result: <a href="https://sv.wiktionary.org/wiki/book#Engelska">book</a> <a href="https://sv.wiktionary.org/wiki/reading#Engelska">reading</a>

Testing ja Wiktionary links for: coffee shop
native url is: https://ja.wiktionary.org/wiki/coffee
native url is: https://ja.wiktionary.org/wiki/shop
Result: <a href="https://ja.wiktionary.org/wiki/coffee#英語">coffee</a> <a href="https://ja.wiktionary.org/wiki/shop#英語">shop</a>
